In [2]:
import torch

#### Junction with 3 incoming and 3 outgoing roads:

In first version assume that all roads have the same maximum density.
Also assume that the priority of connections going into road j is different (or 0) for all connections (i,j).

In [62]:
# Junction with four incoming roads and four outgoing roads:
# Let roads 0, 3 and 4 be incoming roads and roads 1, 2, 5 be outgoing roads

# Let the desired fluxes after multiplying with distribution matrix be:

# fluxes[i,j] is the desired flux from road i to road j
fluxes = [[1.0, 0.0,  0.1],
          [0.0,  0.21, 0.8],
          [0.59, 0.19, 0.7]]

# Let the capacities of the outgoing roads be
original_capacities = [0.4, 0.4, 0.7]
capacities = [0.4, 0.4, 0.7]

# priorities of the roads:
# set equal to 0 if the connection does not exist
priorities = [[1, 0, 1],
              [0, 1, 2],
              [2, 2, 3]]

crossing_connections = [[[], [], []],
                        [[], [], [(0,0)]],
                        [[(0,0), (1,1), (1,2)], [(1,1)], [(0,0), (1,1)]]]

In [63]:
def calculate_upper_bound(crossing_connections, fluxes):
    upper_bound = 1.0
    # Crossing connections on the form
    crossing_fluxes = [fluxes[i][j] for i,j in crossing_connections]
    upper_bound = max(0, 1.0 - sum(crossing_fluxes))
    return upper_bound

In [71]:
fluxes = [[1.0, 0.0,  0.1],
          [0.0,  0.21, 0.8],
          [0.59, 0.19, 0.7]]

# Let the capacities of the outgoing roads be
original_capacities = [0.4, 0.4, 0.7]
capacities = [0.9, 0.4, 0.7]

# priorities of the roads:
# set equal to 0 if the connection does not exist
# the flux at these positions will remain at 0
priorities = [[1, 0, 1],
              [0, 1, 2],
              [2, 2, 3]]

crossing_connections = [[[], [], []],
                        [[], [], [(0,0)]],
                        [[(0,0), (1,1), (1,2)], [(1,1)], [(0,0), (1,1)]]]

entering = [1, 4, 5]
leaving = [2, 3, 6]


max_priority = max([max(row) for row in priorities])
n = 3 # Incoming roads
m = 3 # Outgoing roads
actual_fluxes = [[0.0 for _ in range(m)] for _ in range(n)] # Elements between 0 and 1

for i in range(1, max_priority+1):
    print(f"Assigning fluxes of roads with priority {i}")
    # Go through the priorities
    for j in range(m):
        print(f"Looking for roads with outgoing road {leaving[j]}")
        # Go through the outgoing roads
        if capacities[j] == 0:
            print(f"No capacity left on road\n")
            # No capacity left on this outgoing road
            # Go to next outgoing road
            continue
        for k in range(n):
            # Go through all connections going to the outgoing road
            if priorities[k][j] == i:
                print(f"Road connection {entering[k]} to road {leaving[j]}  has priority {i}")
                # If the priority of the incoming road is equal to the current priority
                # this is the connection that we need to update

                # Calculate the upper bound for this connection:
                if len(crossing_connections[k][j]) == 0:
                    print(f"Connection {entering[k]} to {leaving[j]} has no crossing connections")
                    # If there are no crossing connections, no upper bound needed
                    actual_fluxes[k][j] = min(capacities[j], fluxes[k][j])
                    print(f"Assigned flux: {actual_fluxes[k][j]}")
                    capacities[j] -= actual_fluxes[k][j] # Always greater or equal to 0
                    print(f"Remaining capacity: {capacities[j]}")
                else:
                    print(f"Connection {entering[k]} to {leaving[j]} has crossing connections {crossing_connections[k][j]}")
                    upper_bound = calculate_upper_bound(crossing_connections[k][j],  actual_fluxes)
                    print(f"Upper bound: {upper_bound}")
                    actual_fluxes[k][j] = min(capacities[j], fluxes[k][j], upper_bound)
                    print(f"Assigned flux: {actual_fluxes[k][j]}")
                    capacities[j] -= actual_fluxes[k][j]
                    print(f"Remaining capacity: {capacities[j]}")
        print()


Assigning fluxes of roads with priority 1
Looking for roads with outgoing road 2
Road connection 1 to road 2  has priority 1
Connection 1 to 2 has no crossing connections
Assigned flux: 0.9
Remaining capacity: 0.0

Looking for roads with outgoing road 3
Road connection 4 to road 3  has priority 1
Connection 4 to 3 has no crossing connections
Assigned flux: 0.21
Remaining capacity: 0.19000000000000003

Looking for roads with outgoing road 6
Road connection 1 to road 6  has priority 1
Connection 1 to 6 has no crossing connections
Assigned flux: 0.1
Remaining capacity: 0.6

Assigning fluxes of roads with priority 2
Looking for roads with outgoing road 2
No capacity left on road

Looking for roads with outgoing road 3
Road connection 5 to road 3  has priority 2
Connection 5 to 3 has crossing connections [(1, 1)]
Upper bound: 0.79
Assigned flux: 0.19
Remaining capacity: 2.7755575615628914e-17

Looking for roads with outgoing road 6
Road connection 4 to road 6  has priority 2
Connection 4 to

In [56]:
actual_fluxes

[[0.4, 0.0, 0.1], [0.0, 0.21, 0.6], [0.0, 0.19, 0.0]]

In [57]:
for j in range(m):
    print(f"Actual flux into road {j}: {sum([actual_fluxes[i][j] for i in range(n)])}")
    print(f"Capacity of road {j}: {original_capacities[j]}")
    print(f"Remaining capacity of road {j}: {capacities[j]} \n")

for i in range(n):
    print(f"Total flux out of road {i}: {sum(actual_fluxes[i])} \n")

Actual flux into road 0: 0.4
Capacity of road 0: 0.4
Remaining capacity of road 0: 0.0 

Actual flux into road 1: 0.4
Capacity of road 1: 0.4
Remaining capacity of road 1: 2.7755575615628914e-17 

Actual flux into road 2: 0.7
Capacity of road 2: 0.7
Remaining capacity of road 2: 0.0 

Total flux out of road 0: 0.5 

Total flux out of road 1: 0.8099999999999999 

Total flux out of road 2: 0.19 



### Now testing the most complicated junction in the system with 4 incoming and 4 outgoing roads

In [ ]:
fluxes = [[0.7, 0.0, 0.15, 0.15],
        [0.0, 0.7, 0.15, 0.15],
        [0.15, 0.15, 0.7, 0.0],
        [0.15, 0.15, 0.0, 0.7]]

original_capacities = [0.4, 0.4, 0.7, 0.6]



